In [83]:
from pyspark.sql.functions import lit, col, isnan, when, expr, year, count
from pyspark.sql import SparkSession

In [85]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data Cleansing") \
    .getOrCreate()


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [70]:
# Load the CSV file into a Spark DataFrame
file_path = "2scopus_300_search.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show initial data
df.show()


+-------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+
|Subject|                 URL|          Identifier|               EID|               Title|             Creator|Publication Name|Publication Date|                 DOI|Cited By Count|    Affiliation Name|Affiliation City|Affiliation Country|
+-------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+
|   MULT|https://api.elsev...|SCOPUS_ID:8521031...|2-s2.0-85210310976|Emx2 is an essent...|         Nguyen T.K.|        iScience|      2024-12-20|10.1016/j.isci.20...|             0|University of Not...|      Notre Dame|      United States|
|   MULT|https://api.elsev...|SCOPUS

In [71]:
# Drop rows where any of the specified columns have missing values
columns_to_check = [
    "Subject", "URL", "Identifier", "EID", "Title", 
    "Creator", "Publication Name", "DOI", "Affiliation Name", 
    "Affiliation City", "Affiliation Country"
]

df = df.na.drop(subset=columns_to_check)

# Show the resulting DataFrame
df.show()

# Count the number of rows after dropping
print(f"Number of rows after dropping missing values: {df.count()}")


+-------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+
|Subject|                 URL|          Identifier|               EID|               Title|             Creator|Publication Name|Publication Date|                 DOI|Cited By Count|    Affiliation Name|Affiliation City|Affiliation Country|
+-------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+
|   MULT|https://api.elsev...|SCOPUS_ID:8521031...|2-s2.0-85210310976|Emx2 is an essent...|         Nguyen T.K.|        iScience|      2024-12-20|10.1016/j.isci.20...|             0|University of Not...|      Notre Dame|      United States|
|   MULT|https://api.elsev...|SCOPUS

In [72]:
# Add a new column 'year' with the value 2024
df = df.withColumn("Year", lit(2024))

# Show the updated DataFrame
df.show()

+-------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|Subject|                 URL|          Identifier|               EID|               Title|             Creator|Publication Name|Publication Date|                 DOI|Cited By Count|    Affiliation Name|Affiliation City|Affiliation Country|Year|
+-------+--------------------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|   MULT|https://api.elsev...|SCOPUS_ID:8521031...|2-s2.0-85210310976|Emx2 is an essent...|         Nguyen T.K.|        iScience|      2024-12-20|10.1016/j.isci.20...|             0|University of Not...|      Notre Dame|      United States|2024|
|   MULT|https:/

In [73]:
# Count all unique values in the 'Affiliation Country' column
unique_countries_count = df.select("Affiliation Country").distinct().count()

print(f"Number of unique countries: {unique_countries_count}")


Number of unique countries: 108


In [74]:
# Drop the 'URL' column from the DataFrame
df = df.drop("URL")

# Show the updated DataFrame
df.show()


+-------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|Subject|          Identifier|               EID|               Title|             Creator|Publication Name|Publication Date|                 DOI|Cited By Count|    Affiliation Name|Affiliation City|Affiliation Country|Year|
+-------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|   MULT|SCOPUS_ID:8521031...|2-s2.0-85210310976|Emx2 is an essent...|         Nguyen T.K.|        iScience|      2024-12-20|10.1016/j.isci.20...|             0|University of Not...|      Notre Dame|      United States|2024|
|   MULT|SCOPUS_ID:8521030...|2-s2.0-85210305527|BMAL1 upregulates...|             Zhou X.|        i

In [75]:
# Ensure 'Publication Date' is in a proper date format
df = df.withColumn("Publication Date", col("Publication Date").cast("date"))
print(f"Number of rows before filtering: {df.count()}")

# Drop rows where 'Publication Date' is not in 2024
df = df.filter(year(col("Publication Date")) == 2024)

# Show the filtered DataFrame
df.show()

# Count the number of remaining rows
print(f"Number of rows remaining after filtering: {df.count()}")

Number of rows before filtering: 7544
+-------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|Subject|          Identifier|               EID|               Title|             Creator|Publication Name|Publication Date|                 DOI|Cited By Count|    Affiliation Name|Affiliation City|Affiliation Country|Year|
+-------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|   MULT|SCOPUS_ID:8521031...|2-s2.0-85210310976|Emx2 is an essent...|         Nguyen T.K.|        iScience|      2024-12-20|10.1016/j.isci.20...|             0|University of Not...|      Notre Dame|      United States|2024|
|   MULT|SCOPUS_ID:8521030...|2-s2.0-85210305527|BMAL1 upregul

In [76]:
# Count NULLs in 'Publication Date'
null_pub_date = df.filter(col("Publication Date").isNull()).count()

# Count NULLs in 'Cited By Count'
null_cited_by_count = df.filter(col("Cited By Count").isNull()).count()

print(f"Number of NULLs in 'Publication Date': {null_pub_date}")
print(f"Number of NULLs in 'Cited By Count': {null_cited_by_count}")

Number of NULLs in 'Publication Date': 0
Number of NULLs in 'Cited By Count': 0


In [77]:
df.show()

+-------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|Subject|          Identifier|               EID|               Title|             Creator|Publication Name|Publication Date|                 DOI|Cited By Count|    Affiliation Name|Affiliation City|Affiliation Country|Year|
+-------+--------------------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------+--------------------+----------------+-------------------+----+
|   MULT|SCOPUS_ID:8521031...|2-s2.0-85210310976|Emx2 is an essent...|         Nguyen T.K.|        iScience|      2024-12-20|10.1016/j.isci.20...|             0|University of Not...|      Notre Dame|      United States|2024|
|   MULT|SCOPUS_ID:8521030...|2-s2.0-85210305527|BMAL1 upregulates...|             Zhou X.|        i

In [79]:
# Save DataFrame to CSV
import os

output_path = os.path.abspath("cleaned_308_researches.csv")
df.write.csv(output_path, header=True, mode="overwrite")

print(f"DataFrame saved to {output_path}")


Py4JJavaError: An error occurred while calling o338.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java: